<a href="https://colab.research.google.com/github/murpunk/Programming_2023/blob/main/17_04_23_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метод опорных векторов (Support Vector Machine, SVM)

### 0. Список литературы

[1] Рашка Себастьян, Мирджалили Вахид. Python и машинное обучение: машинное и глубокое обучение с использованием Python, scikit-learn и
TensorFlow 2, 3-е изд.: Пер. с англ. СПб. : ООО "Диалектика", 2020. 848 с.

[2] Плас Дж. Вандер. Python для сложных задач: наука о данных и машинное
обучение. СПб.: Питер, 2018. 576 с.

### I. Прочитайте пункт “Метод опорных векторов” и более подробно о методе опорных векторов в книгах [1, стр. 113–123] или [2, стр. 459–474] и ответьте на вопросы

#### 1. Чем метод отличается от других линейных методов? Какое оптимальное решение он ищет?

В практических задачах линейный SVM и линейная регрессия выдают похожие результаты. Но линейная регрессия больше подвержена выбросам, а SVM работает с опорными векторами, то есть с точками, ближайшими к разделяющей гиперплоскости.

Но линейную регрессию проще реализовать, потому что это более простая модель. Она легко обновляется, что позволяет использовать её с потоковыми данными.

В линейной модели классификации решение ищется в виде функции знака
числа (sign) от суммы всех признаков $f_i(x)$ с коэффициентами $w_i$:

$a(x, w) = sign(\langle x, w \rangle - w_0) = sign\left( \sum\limits_{j=1}^n f_j(x)w_j - w_0\right)$

#### 2. Какие вектора называются опорными?

Опорными векторами называются обучающие образцы, которые расположены ближе всего к гиперплоскости.

#### 3. На что влияет параметр C? Что происходит при его увеличении и уменьшении?

Через параметр C можно управлять штрафом за неправильную классификацию. Чем больше С, тем больше штраф за ошибки. Также C применяется для управления шириной зазора.

Зазор — это расстояние между положительной и отрицательной гиперплоскостями.

#### 4. Для чего используются ядра? Какие ядра есть в библиотеке Scikit-Learn?

Ядра используются для линейно несепарабельных данных. Ядерный SVM способен провести нелинейную границу решений.

В библиотеке Scikit-Learn есть линейное ядро, полиномиальное ядро степени d, радиальное (гауссовское, RBF). По умолчанию используется RBF.

#### 5. Для каких наборов данных лучше использовать метод?

Метод опорных векторов в scikit-learn поддерживают как плотные, так и разреженные выборочные векторы в качестве входных данных. Но с разреженными данными может работать хуже.

#### 6. Какие преимущества и недостатки метода?

**Преимущества**:
*  метод сводится к решению задачи выпуклого квадратичного программирования, которая имеет единственное решение;
*  имеются эффективные численные методы для SVM;
*  выделяется множество опорных объектов (объектов, лежащих вблизи
границ классов);
*  находит оптимальную разделяющую поверхность с максимальным
отступом;
*  обобщается на нелинейные классификаторы;
*  невосприимчивость к тому, как ведут себя удалённые точки;
*  небольшое количество используемой оперативной памяти;
*  приспособляемость к множеству типу данных.

**Недостатки**:
*  опорными векторами могут стать выбросы;
*  необходимо выбирать константу C при помощи кросс-валидации, так как результаты зависят от удачности выбора этого параметра.

### II. Выполните задания из файлов 6_1_statement-svm.pdf и 6_2_statement-svm-texts.pdf

### 6_1_statement.svm.pdf

#### 1. Загрузите выборку из файла svm-data.csv. В нем записана двумерная выборка (целевая переменная указана в первом столбце, признаки — во втором и третьем).

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd # работа с таблицами
import numpy as np  # линейная алгебра: работа с векторами
path_to_data = 'drive/My Drive/Программирование/svm-data.csv'

In [7]:
df = pd.read_csv(path_to_data, header = None)
df

,0,1,2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39
5,1.0,0.34,0.73
6,0.0,0.44,0.06
7,1.0,0.40,0.74
8,0.0,0.18,0.18
9,1.0,0.53,0.53


In [8]:
X = df[0] # выделили целевую переменную из первого столбца
X

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
5    1.0
6    0.0
7    1.0
8    0.0
9    1.0
Name: 0, dtype: float64

In [9]:
y = df.iloc[:, 1:] # выделили признаки из второго и третьего столбцов
y

,1,2
0,0.70,0.29
1,0.23,0.55
2,0.72,0.42
3,0.98,0.68
4,0.48,0.39
5,0.34,0.73
6,0.44,0.06
7,0.40,0.74
8,0.18,0.18
9,0.53,0.53


#### 2. Обучите классификатор с линейным ядром, параметром C = 100000 и random_state=241. Такое значение параметра нужно использовать, чтобы убедиться, что SVM работает с выборкой как с линейноразделимой. При более низких значениях параметра алгоритм будет настраиваться с учетом слагаемого в функционале, штрафующего за маленькие отступы, из-за чего результат может не совпасть с решением классической задачи SVM для линейно разделимой выборки.

In [10]:
from sklearn.svm import SVC

In [11]:
clf = SVC(C = 100000, kernel="linear", random_state = 241)
clf.fit(y, X)

SVC(C=100000, kernel='linear', random_state=241)

#### 3. Найдите номера объектов, которые являются опорными (нумерация с единицы). Они будут являться ответом на задание. Обратите внимание, что в качестве ответа нужно привести номера объектов в возрастающем порядке через запятую. Нумерация начинается с 1.

In [13]:
support_indexes = clf.support_
for i in support_indexes:
  print(i + 1)

4
5
10


### 6_2_statement-svm-texts.pdf

#### 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм".

In [14]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [15]:
newsgroups = datasets.fetch_20newsgroups(
    subset = "all",
    categories = ["alt.atheism", "sci.space"]
)

In [16]:
X_train = newsgroups.target  # целевая переменная
y_train = newsgroups.data    # признаки

In [17]:
X_train

array([0, 0, 1, ..., 1, 1, 0])

In [ ]:
y_train # много непонятных данных

#### 2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки, но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [20]:
tf_idf = TfidfVectorizer()
newsgroups_train = tf_idf.fit_transform(y_train)

#### 3. Подберите минимальный лучший параметр C из множества $[10^{-5},10^{-4}, ..., 10^4,10^5]$ для SVM с линейным ядром (kernel=’linear’) при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [21]:
c = {'C': [10.0 ** x for x in range(-5, 6)]} # параметр регуляризации
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel="linear", random_state=241)
gridsearch = GridSearchCV(clf, c, scoring='accuracy', cv=cv)
gridsearch.fit(newsgroups_train, X_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                               100.0, 1000.0, 10000.0, 100000.0]},
             scoring='accuracy')

In [22]:
c = gridsearch.best_params_.get("C") # минимальный лучший параметр C
c

1.0

#### 4. Обучите SVM по всей выборке с лучшим параметром C, найденным на предыдущем шаге.

In [23]:
svm = SVC(C = c, kernel = "linear", random_state=241)
svm.fit(newsgroups_train, y_train)

SVC(kernel='linear', random_state=241)

#### 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.

In [24]:
terms = tf_idf.get_feature_names_out() # слова из корпуса

In [26]:
import time
from tqdm import tqdm

In [27]:
wights = tqdm(abs(svm.coef_[0])) # показывает веса для слов из корпуса: модуль веса
# работает полчаса, потом завершает работу с ошибкой

KeyboardInterrupt: ignored